## LSSTCam search for low-level persistence.

Run E1586 had images with a series of saturated spots, and one test\
was to follow this with a 100 second dark.  Faint after-images could be seen\
in this long dark with the CCS image viewer.  This notebook analyzes one\
of these images 20241023-165 to try to quantify the amount of persistence.

Craig Lage - 23-Oct-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot

In [ ]:
butler = Butler('embargo_new', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded'])

## First, get a list of exposures

In [ ]:
dayObs = 20241023
instrument = "LSSTCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result
### With the CCS stretch, you can see faint ghosts of the saturated spots.

In [ ]:
%matplotlib inline
expId = 2024102300165
detector = 84  # R21/S10
instrument = 'LSSTCam'
exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
isrResult = isrTask.run(exp) # This runs the ISR
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_{detector}_{expId}.png")

## First, just plot a stripe through the top row of spots.
###  There is too much noise and the spots are not visible

In [ ]:
plt.plot(isrResult.exposure.image.array[2900, :])
plt.xlabel("X pixel")
plt.ylabel("Flux (e-)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/Slice_Thru_Spots_{detector}_{expId}.png")

## Next, try averaging over a 100 pixel height.
### Then the pixels are faintly visible.

In [ ]:
plt.plot(np.median(isrResult.exposure.image.array[2900:3000, :], axis=0))
plt.xlabel("X pixel")
plt.ylabel("Flux (e-)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/Vertically_Averaged_Slice_Thru_Spots_{detector}_{expId}.png")

## Then run a 20 pixel moving average in the X-direction.
### This averages down the noise further and now you can see the spots.

In [ ]:
vertical_average = np.median(isrResult.exposure.image.array[2900:3000, :], axis=0)
width = 20
horizontal_and_vertical_average = np.convolve(vertical_average, np.ones(width), 'valid') / width
plt.plot(horizontal_and_vertical_average)
plt.xlabel("X pixel")
plt.ylabel("Flux (e-)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/Horizontally_and_Vertically_Averaged_Slice_Thru_Spots_{detector}_{expId}.png")